In [155]:
import xml.etree.cElementTree as etree
import re
from tqdm import tqdm
import os
import nltk
from bs4 import BeautifulSoup
from html2text import html2text 
import re
import numpy as np
np.random.seed(7)

In [73]:
def clean_html(html):
    """
    Copied from NLTK package.
    Remove HTML markup from the given string.

    :param html: the HTML string to be cleaned
    :type html: str
    :rtype: str
    """

    # First we remove inline JavaScript/CSS:
    cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", html.strip())
    # Then we remove html comments. This has to be done before removing regular
    # tags since comments can contain '>' characters.
    cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
    # Next we can remove the remaining tags:
    cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
    # Finally, we deal with whitespace
    cleaned = re.sub(r"&nbsp;", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    cleaned = re.sub(r"  ", " ", cleaned)
    return cleaned.strip()


In [81]:
def loadFiles(directory):
    DEBUG =False
    l = []
    for file in os.listdir(directory):
        data = directory + file
        if DEBUG : print (data)
        l.append(data)
    l=sorted(l)
    
    return l

In [90]:
cancer_file_loc = "./Dataset/train/cancer/"
non_cancer_file_loc = "./Dataset/train/noncancer/" 

In [91]:
files_cancer = loadFiles(cancer_file_loc)
files_noncancer = loadFiles(non_cancer_file_loc)

In [93]:
cancer=[]
for i in tqdm(files_cancer):
    xmlDoc = open(i, 'r')
    xmlDocData = xmlDoc.read()
    html=xmlDocData
    cleanhtml = clean_html(html)
    text = html2text(cleanhtml)
    soup = BeautifulSoup(html,"lxml")
    text2 = soup.get_text()
    cancer.append([text2,'c'])

100%|██████████| 300/300 [00:47<00:00,  6.32it/s]


In [99]:
noncancer=[]
for i in tqdm(files_noncancer):
    xmlDoc = open(i, 'r')
    xmlDocData = xmlDoc.read()
    html=xmlDocData
    cleanhtml = clean_html(html)
    text = html2text(cleanhtml)
    soup = BeautifulSoup(html,"lxml")
    text2 = soup.get_text()
    noncancer.append([text2,'nc'])

100%|██████████| 150/150 [00:13<00:00, 11.27it/s]


In [101]:
import pandas as pd

In [103]:
cnT  = pd.DataFrame(cancer,columns=['text','lable'])
NcnT = pd.DataFrame(noncancer,columns=['text','lable'])

In [132]:
cm = [cnT,NcnT]
df = pd.concat(cm,ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

In [138]:
df

,text,lable
0,Cancer ImagingCancer ImagingCICancer ImagingCa...,c
1,Case Rep PsychiatryCase Rep PsychiatryCRIM.PSY...,nc
2,Cancer ImagingCICancer ImagingCancer Imaging17...,c
3,"Cancer Immunol ImmunotherCancer Immunology, Im...",c
4,"Cancer Immunol ImmunotherCancer Immunology, Im...",c
5,Cancer ImagingCICancer Imaging1740-50251470-73...,c
6,Case Rep PsychiatryCase Rep PsychiatryCRIPSCas...,nc
7,Cancer Immunol ImmunotherCancer Immunol. Immun...,c
8,Clin Drug InvestigClin Drug InvestigClinical D...,nc
9,Case Rep PsychiatryCase Rep PsychiatryCRIM.PSY...,nc


# SVM

In [142]:
svm_dataframe          = df
svm_dataframe['lable'] = df['lable'].map({"c":1,"nc":0})

In [143]:
svm_dataframe

,text,lable
0,Cancer ImagingCancer ImagingCICancer ImagingCa...,1
1,Case Rep PsychiatryCase Rep PsychiatryCRIM.PSY...,0
2,Cancer ImagingCICancer ImagingCancer Imaging17...,1
3,"Cancer Immunol ImmunotherCancer Immunology, Im...",1
4,"Cancer Immunol ImmunotherCancer Immunology, Im...",1
5,Cancer ImagingCICancer Imaging1740-50251470-73...,1
6,Case Rep PsychiatryCase Rep PsychiatryCRIPSCas...,0
7,Cancer Immunol ImmunotherCancer Immunol. Immun...,1
8,Clin Drug InvestigClin Drug InvestigClinical D...,0
9,Case Rep PsychiatryCase Rep PsychiatryCRIM.PSY...,0


In [144]:
from sklearn.feature_extraction.text import CountVectorizer

In [148]:
holdtext = np.array(svm_dataframe['text'])

In [174]:
vect = CountVectorizer(max_features=50000)
vect.fit(holdtext)
simple_train_dtm = vect.transform(holdtext)
std   =  simple_train_dtm.toarray()

In [175]:
X = std
y = np.array(svm_dataframe['lable'].astype('int'))

In [176]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

In [177]:
from sklearn import svm

In [178]:
clf = svm.SVC()

In [179]:
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [180]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

In [181]:
result1 = clf.predict(X_test)

In [182]:
confusion_matrix(y_test,result1)

array([[46,  4],
       [ 1, 98]])

In [183]:
accuracy_score(y_test,result1)

0.96644295302013428

# Boosted tree